In [1]:
import simple_antifraud as saf

builder = saf.ModelBuilder()

Создание объекта антифрод системы с различными моделями. 
Параметры:
- `verbose` - вывод транскрипции
- `preprocessor` - зашумление записи перед предсказанием

In [2]:
antifraud = saf.SimpleAntifraud([
    saf.SimpleAntifraudPart(builder.get_regular(), verbose=True),
    saf.SimpleAntifraudPart(builder.get_regular(), preprocessor=True, verbose=True),
    saf.SimpleAntifraudPart(builder.get_retrain('gauss_retrain', 'checkpoints/noised.ckpt'), verbose=True),
    saf.SimpleAntifraudPart(builder.get_retrain('adv_retrain', 'checkpoints/noised_advs.ckpt'), verbose=True),
])

Доступные модели:

In [16]:
antifraud.parts

{'regular': <simple_antifraud.SimpleAntifraudPart at 0x7ffb94477760>,
 'gauss': <simple_antifraud.SimpleAntifraudPart at 0x7ffa7e530e80>,
 'gauss_retrain': <simple_antifraud.SimpleAntifraudPart at 0x7ffa7e4d9f40>,
 'adv_retrain': <simple_antifraud.SimpleAntifraudPart at 0x7ffa7e4a38b0>}

Предсказание обычной записи оригинальной моделью:

In [4]:
antifraud.check_regular('transcription_prepared/normal/112-123215-0030.wav')

(False, ['thankful', 'that', 'one', 'of', "roy's", 'sisters', 'was', 'liable'])

Предсказание мошеннической записи оригинальной моделью:

In [18]:
ans = antifraud.check_regular('transcription_prepared/fraud/cvv_indian.wav')
ans[0], " ".join(ans[1])

(True,
 "hello sir i'm from bank please tell me your can number for a bank purification")

Предсказание мошеннической записи с состязательным шумом оригинальной моделью:

In [20]:
ans = antifraud.check_regular('transcription_prepared/adversarial/cvv_indian.wav')
ans[0], " ".join(ans[1])

(False,
 'hello honey how are you i hope you have a nice weekend with your team mates')

Предсказание мошеннической записи с состязательным шумом защищенной моделью:

In [22]:
ans = antifraud.check_adv_retrain('transcription_prepared/adversarial/cvv_indian.wav')
ans[0], " ".join(ans[1])

(True,
 "hello sir i'm from bank please tell me your can number for a bank purification")

Показать предсказания всех доступных моделей:

In [15]:
for ans in antifraud.check_all('transcription_prepared/adversarial/cvv_indian.wav'):
    print(f'{ans[0]} model prediction: {ans[1][0]}')
    print(f'Transcription: {" ".join(ans[1][1])}')

regular model prediction: False
Transcription: hello honey how are you i hope you have a nice weekend with your team mates
gauss model prediction: False
Transcription: iron short am ally e tell me your same are number for avertigation
gauss_retrain model prediction: True
Transcription: hello sir i'm from bank please tell me your can number for a bank purification
adv_retrain model prediction: True
Transcription: hello sir i'm from bank please tell me your can number for a bank purification
